In [41]:
import time
import random
import numpy as np
import pandas as pd



### Carga de ejemplares

In [ ]:
# Los ejemplares se subieron a un repositorio de Github por lo que para descargarlos y poder usarlos se tiene que ejecutar la siguiente celda:
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/pr152.tsp
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/eil51.tsp
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/ch130.tsp
!wget https://raw.githubusercontent.com/FridaVargas/tsp/main/berlin52.tsp
!wget https://raw.githubusercontent.com/mastqe/tsplib/refs/heads/master/pr76.tsp


--2024-09-19 16:42:58--  https://raw.githubusercontent.com/FridaVargas/tsp/main/pr152.tsp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2206 (2.2K) [text/plain]
Saving to: ‘pr152.tsp.1’

pr152.tsp.1         100%[===================>]   2.15K  --.-KB/s    in 0s      

2024-09-19 16:42:58 (35.2 MB/s) - ‘pr152.tsp.1’ saved [2206/2206]

--2024-09-19 16:42:58--  https://raw.githubusercontent.com/FridaVargas/tsp/main/eil51.tsp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579 [text/plain]
Saving to: ‘eil51.tsp.1’

eil51.tsp.1       

Si lo quieres correr desde VSCode en Windows: 

In [42]:
import requests

urls = [
    "https://raw.githubusercontent.com/FridaVargas/tsp/main/pr152.tsp",
    "https://raw.githubusercontent.com/FridaVargas/tsp/main/eil51.tsp",
    "https://raw.githubusercontent.com/FridaVargas/tsp/main/ch130.tsp",
    "https://raw.githubusercontent.com/FridaVargas/tsp/main/berlin52.tsp",
    "https://raw.githubusercontent.com/mastqe/tsplib/refs/heads/master/pr76.tsp"
]

for url in urls:
    filename = url.split("/")[-1]
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Archivo {filename} descargado.")


Archivo pr152.tsp descargado.
Archivo eil51.tsp descargado.
Archivo ch130.tsp descargado.
Archivo berlin52.tsp descargado.
Archivo pr76.tsp descargado.


### Implementa un algoritmo que lea un archivo de ejemplares para el TSP.

In [43]:
def lectura_TSP(ejemplar, imprimir = False):
  """
  Función que recibe de parámetro un ejemplar de TSP con el formato especificado en TSPLIB y de parámetro opcional si quieres imprimir en pantalla el nombre
  y la dimensión del ejemplar, por default no se imprime.
  Devuelve un diccionario con sus atributos y una tabla
  con los nodos(ciudades) y las coordenadas de dónde están. Además imprime en pantalla el nombre y la dimensión del ejemplar.
  """
  archivo = open(ejemplar,"r")
  contenido = archivo.readlines()
  datos = {} # Aquí se van a guardar los atributos del ejemplar
  linea = 0 # Nos dice cuántas líneas son de especificaciones
  while ':' in contenido[linea]: # Se busca el caracter especial ":" para separar la palara clave del valor
    atributo = contenido[linea].replace('\n', "")
    llave, valor = atributo.split(':',1) # Se guarda en el diccionario
    llave = llave.replace(" ", "")
    datos[llave] = valor
    linea += 1
  datos['lectura'] = linea
  dimension = int(datos['DIMENSION'])
  ciudades = contenido[linea+1:]
  # Para ver la cantidad de coordenadas nos fijamos en el primer set de datos que es el nodo cor1, cor2..
  n = len(ciudades[0].split(" ")) # hay n-1 coordenadas
  listas = [[] for i in range(n)]# Creamos una lista de listas para ir almacenando las coordenadas y nodos para despues pasarlo a un DataFrame
  for i in range(0, dimension):
    ciudad = ciudades[i].replace('\n', "")
    partes = ciudad.split(" ")
    for j in range(0, n):
      listas[j].append(float(partes[j]))
  columnas = ["no. ciudad"] + ["coordenada" + str(i) for i in range(1,n)]
  # Creamos un dataframe
  coordenadas= pd.DataFrame(np.transpose(listas), columns= columnas, index = [i for i in range(1, dimension+1)])
  # Se imprimen en pantalla las caracteristicas del ejemplar si imprimir = True
  if imprimir == True:
    print("El nombre del ejemplar es: ", datos['NAME'], "\n")
    print("La dimensión del ejemplar es: ", datos['DIMENSION'], "\n")
    # Si se desean más especificaciones hay que descomentar las siguientes dos lineas paa que se impriman
    #print("Otras especificaciones: \n")
    #print(datos)
  return datos, coordenadas




Probamos en los 4 ejemplares la implementación:

In [44]:
lectura_TSP("berlin52.tsp", True)
print("------------------------------\n")
lectura_TSP("ch130.tsp", True)
print("------------------------------\n")
lectura_TSP("eil51.tsp", True)
print("------------------------------\n")
lectura_TSP("pr152.tsp", True)
print("------------------------------\n")
lectura_TSP("pr76.tsp", True)
print("------------------------------\n")


El nombre del ejemplar es:   berlin52 

La dimensión del ejemplar es:   52 

------------------------------

El nombre del ejemplar es:   ch130 

La dimensión del ejemplar es:   130 

------------------------------

El nombre del ejemplar es:   eil51 

La dimensión del ejemplar es:   51 

------------------------------

El nombre del ejemplar es:   pr152 

La dimensión del ejemplar es:   152 

------------------------------

El nombre del ejemplar es:   pr76 

La dimensión del ejemplar es:   76 

------------------------------



### Ahora realizamos la matriz de distancias
Empleando el hecho que $d(i,j) = d(j,i)$ reducimos la complejidad computacional del problema. Pues $d(1,j) = d(j,1)$ y así nos queda una matriz de $(n-1) \cdot (n-1)$ luego, $d(2,j) = d(i,2)$ y así sucesivamente. Concluyéndose así que sólo es necesario un ciclo _for_.

In [45]:
def matriz_distancias(ejemplar):
  """
  Función que recibe de parámetro un ejemplar de TSP con el formato especificado en TSPLIB y devuelve su matriz de distancias asociada
  """
  datos, coordenadas = lectura_TSP(ejemplar)
  n = int(datos['DIMENSION'])
  matriz = np.zeros((n,n))
  # Calcula distancias entre cada par de nodos
  for i in range(n):  # Itera sobre cada nodo
      for j in range(i + 1, n):  # Solo itera sobre la mitad trangular superior derecha
          distancia = np.sqrt((coordenadas['coordenada1'].iloc[i] - coordenadas['coordenada1'].iloc[j])**2 +
                              (coordenadas['coordenada2'].iloc[i] - coordenadas['coordenada2'].iloc[j])**2)
          matriz[i][j] = distancia  # Almacena la distancia en la matriz
          matriz[j][i] = distancia  # La distancia es la misma de j a i

  return matriz


### Implementa un algoritmo para generar soluciones aleatorias para el TSP, utilizando algún esquema de codificación basado en permutaciones

In [8]:
# Usamos la funcion permutation de numpy, dejando fijo 1 y permutando de 2 a n
# Convertimos a lista después para un problema en particular, por lo que la llamaremos dentro de la función

'''def generar_sol_aleatoria(ejemplar, semilla =int(time.time())):
  np.random.seed(semilla)
  datos, coordenadas = lectura_TSP(ejemplar)
  dimension = int(datos['DIMENSION'])
  solucion = list([1] + list(np.random.permutation(list(range(2, dimension + 1)))))
  return solucion # Como lista'''

# Intentamos implementar lo anterior, pero generaba la misma solución. Las celdas se ejecutaban muy rápidamente
# y no le daba tiempo de cambiar el valor de la semilla.


"def generar_sol_aleatoria(ejemplar, semilla =int(time.time())):\n  np.random.seed(semilla)\n  datos, coordenadas = lectura_TSP(ejemplar)\n  dimension = int(datos['DIMENSION'])\n  solucion = list([1] + list(np.random.permutation(list(range(2, dimension + 1)))))\n  return solucion # Como lista"

In [46]:
import numpy as np

def generar_sol_aleatoria(ejemplar):
    datos, coordenadas = lectura_TSP(ejemplar)
    dimension = int(datos['DIMENSION'])
    solucion = list([1] + list(np.random.permutation(list(range(2, dimension + 1)))))
    return solucion  # Como lista


In [179]:
#generar_sol_aleatoria('pr76.tsp')


### Implementa un algoritmo para evaluar una solución (permutación)

In [47]:
def evaluar_sol(ejemplar, solucion):
  matriz = matriz_distancias(ejemplar) # llamamos matriz de distancias
  # inicializamos la distancia total
  total = 0
  for i in range(len(solucion)-1):
    a = int(solucion[i])-1 #Hacemos que el index comience en cero
    b = int(solucion[i+1])-1 # Primero los pasamos a enteros para que los reconozca adecuadamente en la matriz
    total += int(matriz[a][b])
  # Falta calcular la distancia del fin de la lista a 1
  c = int(solucion[-1])-1
  total += int(matriz[c][1])

  return total


In [48]:
# Lo probamos para un ejemplar cualquiera de los dados:
solucion = generar_sol_aleatoria('ch130.tsp')
costo = evaluar_sol('ch130.tsp', solucion)
print("El costo de la solucion es", costo)


El costo de la solucion es 45877


### Programa 1
Programa que reciba como parámetros (en la misma línea de ejecución):

 * Nombre del archivo con los datos del ejemplar
 * Semilla del generador de aleatorio
 * Nombre del archivo con los datos de la solución generada

Como salida, deberá imprimir algunos datos generales del ejemplar leído. Por ejemplo:

  * Nombre del ejemplar (archivo)
  * Tamaño del ejemplar (número de ciudades)
  * Arista de mayor peso (o distancia más grande)
  * Ejemplo de solución (solo los primeras y los últimas 3 elementos de la permutación)
  * En caso de que se haya indicado, se deberá escribir en un archivo de texto la solución generada (completa)


In [49]:
def arista_mayor(ejemplar):
  matriz = matriz_distancias(ejemplar)
  n = len(matriz)
  # Inicializamos variables para ahí guardar el no. de columna y no. de fila del maayor valor entre ciudades
  i_mayor = -1
  j_mayor = -1
  valor_mayor = -1

  for i in range(n):
    for j in range(n):
      if float(matriz[i][j]) > valor_mayor:
        i_mayor = i
        j_mayor = j
        valor_mayor = float(matriz[i][j])
  return i_mayor+1, j_mayor+1, valor_mayor # Sumamos 1 para que el index de las ciudades comience en 1 como en los originales


def lectura_ejemplar(ejemplar, nombre_archivo=str(time.time())):
  datos, coordenadas = lectura_TSP(ejemplar, True) # True para imprimir nombre y dimension
  i_mayor, j_mayor, valor_mayor = arista_mayor(ejemplar)
  # Imprimir arista de mayor valor y entre cuales ciudades está
  print("La arista de mayor peso se encuentra entre las ciudades ", i_mayor, "y ", j_mayor, "con un peso de ", valor_mayor,"\n")
  # Ejemplo de sol e imprimimos sus primeros y últimos 3 valores
  solucion = generar_sol_aleatoria(ejemplar)
  print("Una solución al ejemplar es:", solucion[0], solucion[1], solucion[2], ",...,", solucion[-3], solucion[-2], solucion[-1])

  archivo = nombre_archivo + ".txt"
  # Guardamos los datos de nombre, dimension como lo especifica TSPLIB y la solucion en el archivo
  contenido = (
    f"NAME : {datos['NAME']}\n"
    f"DIMENSION: {datos['DIMENSION']}\n"
    f"Una solución al ejemplar es \n"
  )
  contenido += ',\n '.join(str(i) for i in solucion)

  with open(nombre_archivo, 'w') as archivo:
    archivo.write(contenido)

  print('La solucion se guardó en el archivo ', f'{nombre_archivo}.txt')


In [50]:
# Lo probamos con ejemplar cualquiera de los dados
lectura_ejemplar('eil51.tsp', 'archivo_sol')


El nombre del ejemplar es:   eil51 

La dimensión del ejemplar es:   51 

La arista de mayor peso se encuentra entre las ciudades  36 y  40 con un peso de  85.63293758829018 

Una solución al ejemplar es: 1 2 16 ,..., 7 45 43
La solucion se guardó en el archivo  archivo_sol.txt


## Programa 2
Programa reciba como parámetros (en la misma línea de ejecución) :

* Nombre del archivo con los datos del ejemplar
* Nombre del archivo con los datos de la solución a evaluar

Como salida, deberá imprimir algunos datos generales del ejemplar leído. Por ejemplo:

* Nombre del ejemplar (archivo)
* Tamaño del ejemplar (número de ciudades)
* Costo de la solución


In [51]:
def leer_sol(archivo_sol, imprimir = False):
  # Abrimos y leemos el archivo, guardando los datos como en la funcion leer_archivo
  with open(archivo_sol, 'r') as f:
    contenido = f.readlines()
  datos = {} # Aquí se van a guardar los atributos del ejemplar
  linea = 0 # Nos dice cuántas líneas son de especificaciones
  solucion = []
  while ':' in contenido[linea]: # Se busca el caracter especial ":" para separar la palara clave del valor
    atributo = contenido[linea].replace('\n', "")
    llave, valor = atributo.split(':',1) # Se guarda en el diccionario
    llave = llave.replace(" ", "")
    datos[llave] = valor
    linea += 1
  datos['lectura'] = linea
  dimension = int(datos['DIMENSION'])
  ciudades = contenido[linea+1:]
  # Nos deshacemos de salto de linea con strip( ) y de la coma con rstrip()
  solucion = [int(ciudad.strip().rstrip(',')) for ciudad in ciudades]
  if imprimir == True:
    print("Ejemplar: ", datos['NAME'], "\n")
    print("Dimension: ", datos['DIMENSION'], "\n")
    print("Solucion en archivo: ", solucion[0], solucion[1], solucion[2], ",...,", solucion[-3], solucion[-2], solucion[-1])
  return solucion


def lectura_solucion(ejemplar, archivo_sol):
  datos, coordenadas = lectura_TSP(ejemplar, True) # True para imprimir nombre y dimension
  solucion = leer_sol(archivo_sol)
  costo = evaluar_sol('ch130.tsp', solucion)
  print("El costo de la solucion es", costo)




In [20]:
lectura_solucion('ch130.tsp', 'archivo_sol')


El nombre del ejemplar es:   ch130 

La dimensión del ejemplar es:   130 

El costo de la solucion es 14610


# Ejercicio 2

In [52]:
'''Primero implementamos el operador de cambio de dos elementos 
   consecutivos para permutaciones.'''

def operadorCambioConsecutivo(permutacion, indice):
    '''Debe recibir una permutación y sólo necesita de un índice porque
       los elementos a intercambiar son consecutivos (queremos el índice
       del primer elemento).'''
    
    '''Aquí es importante poner `nuevaPermutacion = permutacion[:]` en 
       vez de `nuevaPermutacion = permutacion`. Porque en la segunda ambos
       nombres apuntan a la misma lista en la memoria y si hacemos la 
       primera, `nuevaPermutacion` es una nueva lista independiente de la
       permutación original, así que podemos modificarla sin cambiar a la
       permutación original.'''
    nuevaPermutacion = permutacion[:]

    # Si dimos un índice menor o igual al que corresponde al penúltimo
    # elemento de la permutación...
    if indice < len(permutacion) - 1:
        nuevaPermutacion[indice], nuevaPermutacion[indice + 1] = nuevaPermutacion[indice + 1], nuevaPermutacion[indice]
    # hacemos la permutación consecutiva (porque se puede)

    return nuevaPermutacion # En caso de haber dado un índice no válido
                            # (por ejemplo, el correspondiente al último
                            # elemento de la lista), se retorna una copia
                            # de la permutación original.


In [53]:
'''Ahora implementamos el operador de intercambio de dos elementos
   no consecutivos.'''

def OperadorNoConsecutivo(permutacion, indice1, indice2):
    '''Recibe una permutación y los dos índices que vamos a intercambiar'''

    # Lo mismo que en el anterior, queremos una copia independiente.
    nuevaPermutacion = permutacion[:]

    # Para que se de el intercambio:
        # Los índices deben ser distintos.
        # Ambos tienen que corresponder a elementos que sí estén en la permutación.
    if (indice1 != indice2) and (indice1 < len(permutacion)) and (indice2 < len(permutacion)):
        nuevaPermutacion[indice1], nuevaPermutacion[indice2] = nuevaPermutacion[indice2], nuevaPermutacion[indice1]
    
    return nuevaPermutacion # En caso de haber dado algún índice no válido o
                            # que ambos índices sean iguales, se retorna una copia 
                            # la permutación original.


In [54]:
'''La siguiente es una implementación de un operador que parte a la
   lista en dos y las intercambia. Es parecido al de intercambio de
   dos elementos, sólo que ahora trabajamos con grupos completos.'''

def OperadorInvParticion(permutacion, indice):
    '''Recibe la permutación y el índice donde vamos a hacer el corte.'''

    # Lo mismo que en los anteriores, queremos una copia independiente.
    nuevaPermutacion = permutacion[:]
    
    # Si dimos un índice que corresponde a un elemento distinto al 
    # primero y el último (de otro modo, no tiene sentido hacer 'el
    # corte').
    if 0 < indice < len(permutacion)-1:
        nuevaPermutacion = nuevaPermutacion[indice:] + nuevaPermutacion[:indice]

    return nuevaPermutacion # En caso de haber dado algún índice no válido o
                            # que éste corresponda al primer o último elemento
                            # se retorna la permutación original.


Veamos unos ejemplos de uso:

In [55]:
permutacion = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

print(f"La permutación original es {permutacion}")
#print(len(permutacion))

# Intercambio de consecutivos:
permConsecutiva1 = operadorCambioConsecutivo(permutacion, 3)
permConsecutiva2 = operadorCambioConsecutivo(permutacion, 8)
permConsecutiva3 = operadorCambioConsecutivo(permutacion, 11) 
# El último nos devolverá la permutación original

print(f"Algunas permutaciones obtenidas de hacer el intercambio de elementos consecutivos son:\n"
      f"{permConsecutiva1}, {permConsecutiva2} y {permConsecutiva3}")

# Intercambio de no-consecutivos:
permNoConsecutiva1 = OperadorNoConsecutivo(permutacion, 2, 5)
permNoConsecutiva2 = OperadorNoConsecutivo(permutacion, 5, 9)
permNoConsecutiva3 = OperadorNoConsecutivo(permutacion, 1, 10)
# El último nos devolverá la permutación original

print(f"\nAlgunas permutaciones obtenidas de hacer el intercambio de dos elementos no necesariamente consecutivos son:\n"
      f"{permNoConsecutiva1}, {permNoConsecutiva2} y {permNoConsecutiva3}")

# Inversión de partición:
permInv1 = OperadorInvParticion(permutacion, 5)
permInv2 = OperadorInvParticion(permutacion, 8)
permInv3 = OperadorInvParticion(permutacion, 9)
# El último nos devolverá la permutación original

print(f"\nAlgunas permutaciones obtenidas de usar el operador de inversión de partición son:\n"
      f"{permInv1}, {permInv2} y {permInv3}")


La permutación original es [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Algunas permutaciones obtenidas de hacer el intercambio de elementos consecutivos son:
[1, 2, 3, 5, 4, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 10, 9] y [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Algunas permutaciones obtenidas de hacer el intercambio de dos elementos no necesariamente consecutivos son:
[1, 2, 6, 4, 5, 3, 7, 8, 9, 10], [1, 2, 3, 4, 5, 10, 7, 8, 9, 6] y [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Algunas permutaciones obtenidas de usar el operador de inversión de partición son:
[6, 7, 8, 9, 10, 1, 2, 3, 4, 5], [9, 10, 1, 2, 3, 4, 5, 6, 7, 8] y [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Ejercicio 3

In [56]:
def generar_vecindades_CambioConsecutivo(solucion_inicial, operadorCambioConsecutivo):
    vecindades = []
    n = len(solucion_inicial)
    
    # Esta es la vecindad completa:
    VECINDAD = [operadorCambioConsecutivo(solucion_inicial, i) for i in range(n - 1)]
    
    # Vecindad pequeña: un subconjunto pequeño
    vecindad1 = VECINDAD[:n // 10] # 10% de los intercambios
    vecindades.append(vecindad1) 

    # Vecindad mediana: cambia más pares consecutivos
    vecindad2 = VECINDAD[:n // 5]  # 20% de intercambios
    vecindades.append(vecindad2)

    # Vecindad grande: 
    vecindad3 = VECINDAD[:n // 3]  # 33% de intercambios
    vecindades.append(vecindad3)  

    return vecindades


def generar_vecindades_NoConsecutivo(solucion_inicial, OperadorNoConsecutivo):
    vecindades = []
    n = len(solucion_inicial)

    # Vecindad pequeña: pocos intercambios no consecutivos
    vecindad1 = []
    for _ in range(n // 10):  # 10% de intercambios
        i, j = random.sample(range(n), 2)
        vecindad1.append(OperadorNoConsecutivo(solucion_inicial, i, j))
    vecindades.append(vecindad1)

    # Vecindad mediana: más intercambios no consecutivos
    vecindad2 = []
    for _ in range(n // 5):  # 20% de intercambios
        i, j = random.sample(range(n), 2)
        vecindad2.append(OperadorNoConsecutivo(solucion_inicial, i, j))
    vecindades.append(vecindad2)

    # Vecindad grande: muchos intercambios no consecutivos
    vecindad3 = []
    for _ in range(n // 3):  # 33% de intercambios
        i, j = random.sample(range(n), 2)
        vecindad3.append(OperadorNoConsecutivo(solucion_inicial, i, j))
    vecindades.append(vecindad3)

    return vecindades


def generar_vecindades_InvParticion(solucion_inicial, OperadorInvParticion):
    vecindades = []
    n = len(solucion_inicial)

    # Vecindad pequeña: realiza cortes en pocos lugares
    vecindad1 = [OperadorInvParticion(solucion_inicial, i) for i in range(1, n // 10)]
    vecindades.append(vecindad1)

    # Vecindad mediana: realiza más cortes
    vecindad2 = [OperadorInvParticion(solucion_inicial, i) for i in range(1, n // 5)]
    vecindades.append(vecindad2)

    # Vecindad grande: realiza cortes en muchos más lugares
    vecindad3 = [OperadorInvParticion(solucion_inicial, i) for i in range(1, n // 3)]
    vecindades.append(vecindad3)

    return vecindades


In [57]:
'''Veamos un ejemplo de uso de nuestras funciones:'''

# Permutación inicial
solucionInicial = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

# Generar vecindades usando el operador de cambio consecutivo
vecindadesCambioConsecutivo = generar_vecindades_CambioConsecutivo(solucionInicial, operadorCambioConsecutivo)

# Generar vecindades usando el operador de intercambio no consecutivo
vecindadesNoConsecutivo = generar_vecindades_NoConsecutivo(solucionInicial, OperadorNoConsecutivo)

# Generar vecindades usando el operador de inversión por partición
vecindadesInvParticion = generar_vecindades_InvParticion(solucionInicial, OperadorInvParticion)

# Mostrar resultados
print("Vecindades usando el operador de cambio consecutivo:")
for ind, vecindad in enumerate(vecindadesCambioConsecutivo):
    print(f"Vecindad {ind+1}: {vecindad}")

print("\nVecindades usando el operador de intercambio no consecutivo:")
for ind, vecindad in enumerate(vecindadesNoConsecutivo):
    print(f"Vecindad {ind+1}: {vecindad}")

print("\nVecindades usando el operador de inversión por partición:")
for ind, vecindad in enumerate(vecindadesInvParticion):
    print(f"Vecindad {ind+1}: {vecindad}")


Vecindades usando el operador de cambio consecutivo:
Vecindad 1: [[2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 3, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
Vecindad 2: [[2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 3, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 4, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
Vecindad 3: [[2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 3, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 4, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 3, 4, 6, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 3, 4, 5, 7, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]

Vecindades usando el operador d

In [58]:
def generarVecindadGeneral(solucionInicial, tipoVecindad):
    '''Genera vecindades según el tipo de vecindad solicitado.'''
    
    if tipoVecindad == "CambioConsecutivo":
        return generar_vecindades_CambioConsecutivo(solucionInicial, operadorCambioConsecutivo)
    
    elif tipoVecindad == "NoConsecutivo":
        return generar_vecindades_NoConsecutivo(solucionInicial, OperadorNoConsecutivo)
    
    elif tipoVecindad == "InvParticion":
        return generar_vecindades_InvParticion(solucionInicial, OperadorInvParticion)
    
    else:
        raise ValueError(f"Tipo de vecindad '{tipoVecindad}' no reconocido.")


In [59]:
'''Ejemplo de uso:'''

# Permutación inicial
solucionInicial = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Generar una vecindad de tipo "CambioConsecutivo"
Vecindad = generarVecindadGeneral(solucionInicial, "CambioConsecutivo")
print("Lista creciente de vecindades generadas con Cambio Consecutivo:\n", Vecindad)

#print(len(Vecindad))


Lista creciente de vecindades generadas con Cambio Consecutivo:
 [[[2, 1, 3, 4, 5, 6, 7, 8, 9, 10]], [[2, 1, 3, 4, 5, 6, 7, 8, 9, 10], [1, 3, 2, 4, 5, 6, 7, 8, 9, 10]], [[2, 1, 3, 4, 5, 6, 7, 8, 9, 10], [1, 3, 2, 4, 5, 6, 7, 8, 9, 10], [1, 2, 4, 3, 5, 6, 7, 8, 9, 10]]]


In [60]:
'''Función que, dada una solución y un ejemplar, evalúa su costo total.'''

ejemplar = "berlin52.tsp" # La idea es poder  mover esto.

def evaluar(s):
    costo_total = evaluar_sol(ejemplar, s)
    return costo_total


In [62]:
'''Función que, dada una solucion inicial y un listado de vecinos, nos escupe el primer vecino en mejorar'''

def primVecinoMejorar(solucionInicial, vecinos):
    mejorVecino = solucionInicial # En principio, la solución inicial ya puede ser el mejor valor de la 
                                  # vecindad.
    mejorEvaluacion = evaluar(solucionInicial) # La solución inicial es, en principio, la mejor 
                                               # evaluación.

    while vecinos: # Mientras haya vecinos en la vecindad
        vecino = random.choice(vecinos) # Elige un vecino al azar
        evaluacionVecino = evaluar(vecino) # Evaluamos al vecino.

        if evaluacionVecino < mejorEvaluacion: # Si mejora la evaluación.
            mejorVecino = vecino
            mejorEvaluacion = evaluacionVecino
            break # Aquí terminamos, buscamos el primer vecino en mejorar.

        vecinos.remove(vecino) # Eliminamos el vecino ya evaluado de la lista
    
    return mejorVecino


In [63]:
'''Función que define una sencilla perturbación estocástica dada una cierta solución.'''

def perturbacion_estocastica(s):
    i, j = random.sample(range(len(s)), 2)
    s[i], s[j] = s[j], s[i]
    return s


In [64]:
# La función recibe como parámetros una solución inicial y un tipo de vecindad determinado 
# por el operador que querramos usar.
def busqueda_por_vecindades(solucionInicial, tipoVecindad):
    mejorSolucion = solucionInicial # En principio, la mejor solución, es la inicial

    itersMAX = 1000
    numVecindad = 0 # Tenemos 3 vecindades, así Vecindades[0, 1, 2]
    itersSinMejorarMAX = 100 # Máximo número de iteraciones sin mejora permitidas
    itersSinMejorar = 0 # Contador de iteraciones sin mejorar
    iteraciones = 0 # Contador de iteraciones
    perturbaciones = 0
    perturbacionesMAX = 5

    while (iteraciones < itersMAX) and (itersSinMejorar < itersSinMejorarMAX) and (perturbaciones < perturbacionesMAX):
        
        Vecindad = generarVecindadGeneral(mejorSolucion, tipoVecindad)
        # 'Vecindad' es una lista de listas, contiene a cada una de las vecindades en orden 
        # creciente, sus elementos están enumerados como: 0, 1, 2
        
        # Queremos que, siempre que podamos mejorar nos sigamos moviendo por las vecindades de una misma 
        # solución y sólo perturbar cuando ya hayamos llegado a un óptimo local.
        while numVecindad <= (len(Vecindad)-1):
            iteraciones += 1 # Empezamos una iteración cada vez que vamos a hacer una exploración en una
                             # vecindad.
            
            vecinos = Vecindad[numVecindad] # El listado de vecinos queda determinado por la vecindad en 
                                            # que vamos.
            
            # Dada la mejor solución hasta ahora, calculamos el primer vecino en mejorar y lo guardamos como 
            # mejor vecino.
            mejorVecino = primVecinoMejorar(mejorSolucion, vecinos) 
            
            # Si logramos mejorar:
            if evaluar(mejorVecino) < evaluar(mejorSolucion):
                mejorSolucion = mejorVecino # El vecino que mejoró es ahora la mejor solución.
                numVecindad = 0 # Seguimos la búsqueda en la vecindad más pequeña.
                itersSinMejorar = 0 # Reiniciamos el contador de las iteraciones sin mejora.

            # Si no logramos mejorar:
            else:    
                itersSinMejorar += 1 # Tuvimos una iteración sin mejorar.
                numVecindad += 1 # Seguimos la búsqueda en una vecindad más grande.

        # Ya no pudimos mejorar con las vecindades dadas por `mejorSolucion`, la vamos a perturbar con la
        # esperanza de poder salir del óptimo local:
        s = perturbacion_estocastica(mejorSolucion)
        perturbaciones += 1
        itersSinMejorar += 1 # Puede que esto lo tengamos que hacer varias veces (perturbar) y las veces que
                             # lo hagamos, lo contaremos como una iteración sin mejorar.
        
        # Si logramos salir del óptimo local por medio de la perturbación:
        if evaluar(s) < evaluar(mejorSolucion):
            mejorSolucion = s # Ahora guardamos a la solución perturbada como `mejorSolucion`
            numVecindad = 0 # Aunque aún no calculamos la vecindad de la nueva `mejorSolucion`, reiniciamos el
                            # contador para poder así volver a generar vecindades y explorarlas para encontrar
                            # mejores soluciones.
    mejorValor = evaluar(mejorSolucion) # Obtenemos el mejor valor que está dado por la mejor solución
    return mejorSolucion, mejorValor
        
        

In [36]:
'''Vamos a hacer un ejemplo de uso para el ejemplar 'pr152.tsp' '''
'''ejemplar = 'pr152.tsp'
solucionInicial = generar_sol_aleatoria(ejemplar)
costoInicial = evaluar(solucionInicial)
print(costoInicial)

mejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "CambioConsecutivo")
print(len(mejorSolucion), mejorValor)'''


'ejemplar = \'pr152.tsp\'\nsolucionInicial = generar_sol_aleatoria(ejemplar)\ncostoInicial = evaluar(solucionInicial)\nprint(costoInicial)\n\nmejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "CambioConsecutivo")\nprint(len(mejorSolucion), mejorValor)'

In [37]:
'''Vamos a hacer un ejemplo de uso para el ejemplar 'pr152.tsp' '''
''''ejemplar = 'pr152.tsp'
solucionInicial = generar_sol_aleatoria(ejemplar)
#costoInicial = evaluar(solucionInicial)
print(costoInicial)

mejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "InvParticion")
print(len(mejorSolucion), mejorValor)'''


'\'ejemplar = \'pr152.tsp\'\nsolucionInicial = generar_sol_aleatoria(ejemplar)\n#costoInicial = evaluar(solucionInicial)\nprint(costoInicial)\n\nmejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "InvParticion")\nprint(len(mejorSolucion), mejorValor)'

In [38]:
'''Vamos a hacer un ejemplo de uso para el ejemplar 'pr152.tsp' '''
'''ejemplar = 'pr152.tsp'
solucionInicial = generar_sol_aleatoria(ejemplar)
costoInicial = evaluar(solucionInicial)
print(costoInicial)

mejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "NoConsecutivo")
print(len(mejorSolucion), mejorValor)'''


'ejemplar = \'pr152.tsp\'\nsolucionInicial = generar_sol_aleatoria(ejemplar)\ncostoInicial = evaluar(solucionInicial)\nprint(costoInicial)\n\nmejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "NoConsecutivo")\nprint(len(mejorSolucion), mejorValor)'

In [39]:
'''Vamos a hacer un ejemplo de uso para el ejemplar 'pr76.tsp' '''
'''ejemplar = 'pr76.tsp'
solucionInicial = generar_sol_aleatoria(ejemplar)
#costoInicial = evaluar(solucionInicial)
#print(costoInicial)

mejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "InvParticion")
print(len(mejorSolucion), mejorValor)'''


'ejemplar = \'pr76.tsp\'\nsolucionInicial = generar_sol_aleatoria(ejemplar)\n#costoInicial = evaluar(solucionInicial)\n#print(costoInicial)\n\nmejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "InvParticion")\nprint(len(mejorSolucion), mejorValor)'

In [65]:
'''Vamos a hacer un ejemplo de uso para el ejemplar 'pr76.tsp' '''
ejemplar = 'pr76.tsp'
solucionInicial = generar_sol_aleatoria(ejemplar)
costoInicial = evaluar(solucionInicial)
print(costoInicial)

mejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "InvParticion")
print(len(mejorSolucion), mejorValor)


561436
76 549037


In [84]:
# Vamos a usar las vecindades dadas por el operador `InvParticion` porque para ejemplares muy grandes
# ('pr152.tsp') es el que menos tarda en realizar una ejecución.
tipoVecindad = "InvParticion"

ejemplares = ['berlin52.tsp', 'ch130.tsp', 'eil51.tsp', 'pr152.tsp', 'pr76.tsp'] 
ejecuciones = 10
dimensiones = {'berlin52.tsp': 52, 'ch130.tsp': 130, 'eil51.tsp': 51, 'pr152.tsp': 152, 'pr76.tsp': 76}

#Para que la tabla se vea linda, le agregamos líneas horizontales que dividen los renglones:
# Función para imprimir una línea horizontal
def imprimir_linea(ancho):
    print('-' * ancho)

ancho_total = 24 + 15 + 20 + 20 + 20  # Ancho total de la tabla


# Ejecutar 25 veces búsqueda aleatoria para cada combinación de dimensión e iteración:
imprimir_linea(ancho_total)
print(f"{'Nombre del ejemplar':<24} {'Dimensión':<15} {'Mejor valor':<20} {'Valor promedio':<20} {'Peor valor':<20}")
imprimir_linea(ancho_total)
# Hasta la línea anterior, sólo es el encabezado de la tabla.

for ejemplar in ejemplares:
    dimension = dimensiones[ejemplar]
    lista = [] # Vamos a definir una lista donde guardar el mejor valor de cada ejecución
    for i in range(0, ejecuciones): # Para cada ejemplar hacemos 10 ejecuciones.
        solucionInicial = generar_sol_aleatoria(ejemplar)
        mejorSolucion, mejorValor = busqueda_por_vecindades(solucionInicial, "InvParticion")
        lista.append(mejorValor)
    valorPromedio = sum(lista)/len(lista)
    mejorDeLosMejores = min(lista)
    peorDeLosMejores = max(lista)

    print(f"{ejemplar:<24} {dimension:<15} {mejorDeLosMejores:<20} {valorPromedio:<20} {peorDeLosMejores:<20}")
    imprimir_linea(ancho_total)


---------------------------------------------------------------------------------------------------
Nombre del ejemplar      Dimensión       Mejor valor          Valor promedio       Peor valor          
---------------------------------------------------------------------------------------------------
berlin52.tsp             52              27452                29076.5              30651               
---------------------------------------------------------------------------------------------------


HASTA AQUÍ 